In [1]:
# Block 1: Importing Libraries and Defining Remove Accentuation Function

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import yaml
import os
import numpy as np
import unicodedata
import re
import json

print("Libraries imported successfully.")

def remove_accentuation(text):
    if isinstance(text, str):
        return ''.join(char for char in unicodedata.normalize('NFKD', text) if not unicodedata.combining(char))
    return text

print("Accent removal function defined successfully.")

Libraries imported successfully.
Accent removal function defined successfully.


In [2]:
# Block 2: Loading YAML Settings

def load_configurations(filepath):
    with open(filepath, 'r') as file:
        config = yaml.safe_load(file)
    return config

# Load settings from the YAML file
yaml_file = 'competencias.yaml'
config = load_configurations(yaml_file)
if config:
    print("YAML configuration loaded successfully.")
    print(json.dumps(config, indent=2))
else:
    print("Failed to load YAML configuration.")

YAML configuration loaded successfully.
{
  "avaliacao": {
    "pessoa_avaliada": {
      "identificador_pergunta": "Sua relacao com {{nome}} e como:"
    }
  },
  "competencias": [
    {
      "nome": "Desenvolvimento de Pessoas",
      "descricao": "Planeja e apoia o desenvolvimento das habilidades de todas as pessoas de sua equipe para que possam atender aos objetivos atuais e futuros de seus PAs. A pessoa demonstra essa compet\u00eancia: 1. Atribuindo responsabilidades e tarefas \u00e0 sua equipe de acordo com suas habilidades, reconhecendo e valorizando os pontos fortes individuais; 2. Trabalhando com as pessoas de sua equipe para desenvolver seus pontos fortes, ajud\u00e1-las a gerenciar suas fraquezas, entender a necessidade pr\u00e1tica de melhoria individual e definir metas de treinamento espec\u00edficas; 3. Reconhecendo conquistas individuais e coletivas e promovendo um clima de autoconfian\u00e7a por meio do bom uso do feedback; 4. Garantindo que oportunidades de treinament

In [3]:
# Block 3: Loading and Processing Excel Data

def load_treat_data(filepath, config):
    df = pd.read_excel(filepath)
    
    # Process columns to be dropped
    for col in config['xlsx_processing']['discard_columns']:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
    
    # Remove accent, if configured
    if config['xlsx_processing']['remove_accentuation']:
        df.columns = df.columns.map(remove_accentuation)
        for col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].map(lambda x: remove_accentuation(x) if isinstance(x, str) else x)
    
    # Convert processed data to JSON
    dataset_json = df.to_json(orient='records', force_ascii=False)
    return json.loads(dataset_json)

# Ask the user for the Excel file name
excel_file = input("Enter the name of the Excel file (without extension): ")
excel_file = excel_file.strip() + '.xlsx'

# Check if the Excel file exists
if not os.path.isfile(excel_file):
    print(f"Excel file not found: {excel_file}")
else:
    # Load and process the Excel data
    dataset_json = load_treat_data(excel_file, config)
    if dataset_json:
        print("Excel data loaded and processed successfully.")
        print(json.dumps(dataset_json, ensure_ascii=False, indent=2))
    else:
        print("Failed to load and process Excel data.")

Excel data loaded and processed successfully.
[
  {
    "Sua relacao com RODRIGO CRAVEIRO e como:": "diretor",
    "Com que frequencia o seu gestor esclarece as expectativas de comportamento, conhecimento e habilidades de todos integrantes da equipe, incluindo voce (pontos fortes e pontos fracos)?": null,
    "Com que frequencia o seu gestor da instrucoes e modelos positivos para desenvolver as habilidades de todos os integrantes da sua equipe, incluindo voce (pontos fortes)?": null,
    "Com que frequencia o seu gestor da feedback positivo e apropriado sobre o desempenho de todos os integrantes da sua equipe, incluindo voce?": null,
    "Com que frequencia o seu gestor oferece suporte aos esforcos e progressos de todos os integrantes da sua equipe, incluindo voce (feedback)?": null,
    "Com que frequencia o seu gestor faz com que todos os integrantes da sua equipe, incluindo voce, se sentirem valorizados e respeitados?": null,
    "Com que frequencia o seu gestor ensina novos procedi

In [4]:
# Block 4: Extracting Person's Name, Creating Output Directory, and Saving JSON File

def extract_evaluated_person_name(dataset_json):
    prefix = "Sua relacao com "
    suffix = " e como:"
    for record in dataset_json:
        for key in record:
            if key.startswith(prefix) and suffix in key:
                person_name = key[len(prefix):key.index(suffix)].strip()
                return person_name
    return "Name not found"

def create_output_directory(person_name):
    """Create an output directory with the name of the person being evaluated."""
    output_dir = f"output_{person_name}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Output directory '{output_dir}' created successfully.")
    else:
        print(f"Output directory '{output_dir}' already exists.")
    return output_dir

def save_json_file(json_data, output_dir, person_name):
    """Save the JSON data to a file in the output directory."""
    json_file_path = os.path.join(output_dir, f"{person_name}_data.json")
    with open(json_file_path, 'w') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=2)
    print(f"JSON data saved successfully to '{json_file_path}'.")

# Extract the name of the person being evaluated
person_name = extract_evaluated_person_name(dataset_json)
print(f"Person being evaluated: {person_name}")

# Create the output directory
output_dir = create_output_directory(person_name)

# Save the JSON data to a file in the output directory
save_json_file(dataset_json, output_dir, person_name)

Person being evaluated: RODRIGO CRAVEIRO
Output directory 'output_RODRIGO CRAVEIRO' created successfully.
JSON data saved successfully to 'output_RODRIGO CRAVEIRO/RODRIGO CRAVEIRO_data.json'.
